In [ ]:
import numpy as np
import pandas as pd
import sqlite3
from sqlite3 import Error

import texthero as hero
from texthero import preprocessing
from sentence_transformers import SentenceTransformer, util
import nmslib

import time
import datetime

In [ ]:
conn = sqlite3.connect("wine_data.sqlite")
c = conn.cursor()

In [ ]:
sql = '''
select * from wine_data
'''

In [ ]:
df = pd.read_sql(sql, conn)

In [ ]:
df.head()

In [ ]:
distilbert = SentenceTransformer('distilbert-base-uncased')
embeddings = distilbert.encode(df['description'], convert_to_tensor=True)
df['distilbert'] = np.array(embeddings).tolist()
# df['pdisroberta']= vect_to_df

# initialize a new index, using a HNSW index on Cosine Similarity
distilbert_index = nmslib.init(method='hnsw', space='cosinesimil')
distilbert_index.addDataPointBatch(embeddings)
distilbert_index.createIndex({'post': 2}, print_progress=True)

In [ ]:
df.head(1)

In [ ]:
print(len(embeddings[0]))
print(embeddings[0])

In [ ]:
def search_wine(dataframe, userQuery):
    
    if dataframe is not None and userQuery is not None:
        df = dataframe.copy()
        query = distilbert.encode([userQuery], convert_to_tensor=True)
        ids, distances = distilbert_index.knnQuery(query, k=20)

        matches = []

        for i,j in zip(ids,distances):

            matches.append({'country':df.country.values[i]
                        , 'winery' : df.winery.values[i]
                        , 'title' : df.title.values[i]
                        , 'variety': df.variety.values[i]
                        , 'color' : df.color.values[i]
                        , 'description': df.description.values[i]
                        , 'price': df.price.values[i]
                        , 'rating': df.rating.values[i]
                        , 'distance': j
                       })

        return pd.DataFrame(matches) 

In [ ]:
search_wine(df, "sweet and tangy")

In [ ]:
def search_wine(dataframe, userQuery, color = None, price = None):
    
    if dataframe is not None and userQuery is not None:

        df = dataframe.copy()
        query = distilbert.encode([userQuery], convert_to_tensor=True)
        e = np.vstack(query)
        ids, distances = distilbert_index.knnQuery(e, k=100)

        matches = []

        for i,j in zip(ids,distances):

            matches.append({'country':df.country.values[i]
                            , 'winery' : df.winery.values[i]
                            , 'title' : df.title.values[i]
                            , 'variety': df.variety.values[i]
                            , 'color' : df.color.values[i]
                            , 'description': df.description.values[i]
                            , 'price': df.price.values[i]
                            , 'rating': df.rating.values[i]
                            , 'distance': j
                           })

        results = pd.DataFrame(matches) 
        
        if color is None:
            color = ''
        
        if price is None:
            price = 999999.999
        
        if len(color) > 0 and len(str(price))>0:
            results = results[(results['color']==color) & (results['price'] <= price) ]
        elif len(color) == 0 and len(str(price))>0:
            results = results[(results['price'] <= price)]
        else:
            results
                      
        results = results.reset_index()
        results = results[['country','winery','title','color','description','price','rating','simRank']]
        return results

In [ ]:
search_wine(df, 'sweet and tangy', 'red', 99.9)

In [ ]:
df['tsnedistilbert'] = hero.tsne(df['distilbert'])

In [ ]:
#Save a meta index and the data
distilbert_index.saveIndex('distilBertWineindex.bin', save_data=True)
#Re-intitialize the library, specify the space
#newIndex = nmslib.init(method='hnsw', space='cosinesimil_sparse')
#Re-load the index and the data
#newIndex.loadIndex('sparse_index.bin', load_data=True)

In [ ]:
#create scatter plot of uni. sent. enc.
hero.scatterplot(df, col='tsnedistilbert'
                 , color='variety'
                 , title="Wine Explorer"
                 , hover_data = ['title','variety','price','description'])

In [ ]:
import dash
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output

In [ ]:
external_stylesheets=[dbc.themes.BOOTSTRAP]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

search_card = dbc.Card(
    dbc.CardBody(
        [
            html.H5("Auto-Sommelier", className="card-title")
            , dbc.Textarea(id = 'user-input', bs_size="lg"
                     , placeholder="Describe Ideal Wine")
             ,dbc.Button('search', id = 'search', color="danger")
        ]
    )
)

app.layout = html.Div([html.H1("Find Your Wine!")
                   , html.Br()
                   , dbc.Row(dbc.Col(search_card,
                        width={"size": 6, "offset": 3}))#end row
                  , html.Br()
                  , dbc.Card(dbc.CardBody(html.Div(id = 'results')))
                  ])

@app.callback(Output("results", "children")
              , [Input("user-input", "value")
                , Input("search", "n_clicks")
                ])
def return_wine(userquery, n):
    
    if userquery is not None and n is not None:
        
        dff = df.copy()
        results = search_wine(dff, userquery)
        table = dbc.Table.from_dataframe(results, striped=True, bordered=True, hover=True)
    
        return table

In [ ]:
app.run_server()